# 处理HCI原始数据成符合可供训练的形式

具体为封装成为pkl形式，参考于Raven的构建

'''

dict_keys(['label', 'subject_list', 'ch_info', 'info', 'eye_info', 'eeg', 'eye', 'au'])
EEG: 1-70filer, 50Hz notch, With ICA, 256Hz resample;
Subject : 1-34 subject, no 1,23,32 subject, 15 subject exists eye data missing; 31 person, 48 question, 31 channel;
Labels : 0: Confused,1: Guess, 2:Unconfused, 4: Think-right;
Eye Track data : Pupil diameter left, Pupil diameter right,Gaze point X, Gaze point Y, Eye movement type, Gaze event duration
['Fp1', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2']
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34]

'''


In [1]:

import os
import sys
import yaml
from pathlib import Path
import pandas as pd
import numpy as np
import joblib
def add_project_root_to_sys_path():
    """动态添加项目根目录到 sys.path"""
    project_root = Path.cwd().resolve().parent.parent
    if str(project_root) not in sys.path:
        sys.path.append(str(project_root))

add_project_root_to_sys_path()

from raw_data_process.hci import data_read as hdr

In [2]:
data = hdr.get_subject_data(1, nb_classes=2)
print(data)

([array([[-1.59061489e-05, -1.21536423e-05, -7.95252077e-06, ...,
         5.82895167e-06,  1.19879893e-06,  2.67090892e-06],
       [-1.67992273e-05, -1.22281194e-05, -8.26652143e-06, ...,
         2.54642965e-06,  1.94729689e-07,  1.00861388e-06],
       [-1.36440781e-05, -9.41968952e-06, -6.65558506e-06, ...,
         1.56241367e-06, -4.02281626e-07, -7.67003303e-07],
       ...,
       [ 5.38937036e-06,  7.04404964e-06,  7.95300290e-06, ...,
        -3.40974346e-06, -1.56704566e-06, -3.81470699e-06],
       [ 6.72968593e-06,  8.98891378e-06,  8.85955379e-06, ...,
        -2.75689907e-06, -1.51800428e-06, -3.04495468e-06],
       [ 6.32713411e-06,  8.37558858e-06,  8.24587602e-06, ...,
        -4.29820942e-06, -1.08816594e-06, -1.78719272e-06]]), array([[-1.03085064e-06, -1.83445864e-06,  3.97736672e-06, ...,
        -7.18442727e-06, -4.10350134e-06, -2.84489054e-06],
       [-2.67717847e-06, -3.12618345e-06,  2.99045128e-06, ...,
        -6.63853174e-06, -3.03374133e-06, -2.4104524

In [8]:
subject_lists = [1, 2, 4, 5, 6, 7, 8, 10, 11, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30]
eeg_data = []
pps_data = []
eye_data = []
arousal_labels = []
valence_labels = []
for sub in subject_lists:
    eeg_trials,pps_trials,eye_trials,arousal_label,valence_label  = hdr.get_subject_data(sub, nb_classes=3)
    print(len(eeg_trials))
    print(eeg_trials[0].shape)
    eeg_data.append(eeg_trials)
    pps_data.append(pps_trials)
    eye_data.append(eye_trials)
    arousal_labels.append(arousal_label)
    valence_labels.append(valence_label)

20
(15116, 32)
20
(22032, 32)
20
(13840, 32)
20
(29988, 32)
20
(15121, 32)
20
(25257, 32)
20
(18475, 32)
20
(13836, 32)
20
(18186, 32)
20
(18178, 32)
20
(15112, 32)
20
(8965, 32)
20
(23023, 32)
20
(22024, 32)
20
(24841, 32)
20
(29985, 32)
20
(15112, 32)
20
(25265, 32)
20
(18463, 32)
20
(13832, 32)
20
(11052, 32)
20
(18181, 32)
20
(22305, 32)
20
(22544, 32)


In [10]:
print(len(eeg_data))
print(len(pps_data))
print(len(eye_data))
print(arousal_labels)
print(valence_labels)

24
24
24
[[0, 2, 1, 1, 0, 1, 1, 1, 2, 1, 2, 0, 2, 0, 1, 0, 0, 1, 0, 1], [1, 0, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 1, 0, 0, 2, 1, 2, 2, 2], [1, 1, 2, 0, 2, 2, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 2, 0, 2, 0], [0, 2, 1, 1, 1, 2, 2, 2, 2, 1, 0, 1, 1, 2, 2, 1, 1, 0, 1, 0], [0, 1, 0, 2, 1, 2, 1, 0, 0, 1, 2, 2, 0, 1, 0, 0, 2, 0, 1, 1], [1, 2, 2, 2, 0, 0, 0, 1, 2, 2, 0, 0, 2, 0, 0, 0, 2, 1, 0, 0], [2, 0, 1, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 2, 2, 1, 2, 1, 1, 0], [0, 1, 2, 2, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 2], [2, 0, 1, 0, 2, 0, 2, 1, 2, 1, 1, 0, 2, 2, 0, 0, 0, 2, 1, 0], [2, 1, 1, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 0], [1, 0, 1, 1, 0, 2, 0, 2, 1, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 1], [2, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 2, 0, 2, 2, 0, 2, 1], [0, 1, 2, 0, 0, 0, 1, 0, 0, 1, 1, 2, 1, 2, 0, 1, 0, 0, 1, 2], [1, 0, 0, 2, 1, 0, 0, 1, 0, 2, 0, 2, 0, 1, 1, 1, 0, 2, 2, 1], [1, 0, 1, 0, 0, 1, 2, 2, 2, 0, 1, 1, 0, 1, 1, 2, 1, 0, 0, 0], [0, 0, 1, 1, 1, 2, 0, 0, 2, 0, 0, 1, 2, 2, 0, 1, 1, 0, 1, 0]

In [12]:
ch_info = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2', 'EXG1', 'EXG2', 'EXG3', 'GSR1', 'Resp', 'Temp', 'Status']
print(len(ch_info))

39


In [11]:
# 整合上述信息保存成pkl
ch_info = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2', 'EXG1', 'EXG2', 'EXG3', 'GSR1', 'Resp', 'Temp', 'Status']
data = {
    "eeg": eeg_data,
    "eye": eye_data,
    "pps": pps_data,
    "arousal_label": arousal_labels,
    "valence_label": valence_labels,
    "subject_list": subject_lists,
    "ch_info": ch_info,
    "info": "EEG: 1-70filer, 50Hz notch, 256Hz resample;\n\
Subject : 1-30 subject, no 3,9,12,15,16,25 subject; 24 person, 20 trial, 32 EEG channel, 7 pps channels;\n\
Labels :    Arousal: 0:[\"Sadness\", \"Disgust\", \"Neutral\"]、1:[\"Joy, Happiness\", \"Amusement\"] 2:[\"Surprise\", \"Fear\", \"Anger\", \"Anxiety\"]\n\
    Valence: 0:[\"Fear\", \"Anger\", \"Disgust\", \"Sadness\", \"Anxiety\"] 1:[\"Surprise\", \"Neutral\"] 2:[\"Joy, Happiness\", \"Amusement\"]\n\
Eye Track data : DistanceLeft，PupilLeft，ValidityLeft，Distance Right，Pupil Right，Validity Right，Fixation Index，Gaze Point X，Gaze Point Y，Fixation Duration\n\
     PPS data : ECG, GSR，Resp，Temp，Status",
}

hci_data_pkl = "/data/MAHNOB/hci_data.pkl"
joblib.dump(data, hci_data_pkl)

['/data/MAHNOB/hci_data.pkl']

In [16]:
# 保存早已处理好的de特征至pkl
hci_de_features_path = "/data/MAHNOB/ExtractedFeatures2/node_features_combine.npy"
hci_de_features = np.load(hci_de_features_path)
print(hci_de_features.shape)

# 保存成pkl
hci_de_features_pkl = "/data/MAHNOB/de_features.pkl"
joblib.dump(hci_de_features, hci_de_features_pkl)

(15540, 585)


['/data/MAHNOB/de_features.pkl']

In [8]:
# 看看眼动和pps特征
subject_lists = [1, 2, 4, 5, 6, 7, 8, 10, 11, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30]
hci_eye_features_path = "/data/MAHNOB/eye_track_feature/{}.npy"
hci_pps_features_path = "/data/MAHNOB/pps_features/{}.npy"

eye_feature = np.load(hci_eye_features_path.format(1))
print(eye_feature.shape)

pps_feature = np.load(hci_pps_features_path.format(1), allow_pickle=True).astype(np.float32)
print(pps_feature.shape)

(20, 38)
(20, 230)


In [9]:
eye_features = []
pps_features = []

for subject in subject_lists:
    eye_feature = np.load(hci_eye_features_path.format(subject))
    pps_feature = np.load(hci_pps_features_path.format(subject), allow_pickle=True).astype(np.float32)
    eye_features.append(eye_feature)
    pps_features.append(pps_feature)

eye_features = np.array(eye_features)
pps_features = np.array(pps_features)

print(eye_features.shape)
print(pps_features.shape)

(24, 20, 38)
(24, 20, 230)


In [11]:
# label
arousal_label_path = "/data/MAHNOB/ExtractedFeatures2/arousal_label.npy"
valence_label_path = "/data/MAHNOB/ExtractedFeatures2/valence_label.npy"

arousal_label = np.load(arousal_label_path)
valence_label = np.load(valence_label_path)

print(arousal_label.shape)
print(valence_label.shape)
print(set(arousal_label))
print(set(valence_label))

(420,)
(420,)
{0, 1, 2}
{0, 1, 2}


In [14]:
print(subject_lists)
print(len(subject_lists))

[1, 2, 4, 5, 6, 7, 8, 10, 11, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30]
24


In [18]:
# 整合上述信息保存成pkl
ch_info = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2', 'EXG1', 'EXG2', 'EXG3', 'GSR1', 'Resp', 'Temp', 'Status']
data = {
    "eeg": hci_de_features,
    "eye": eye_features,
    "pps": pps_features,
    "arousal_label": arousal_label,
    "valence_label": valence_label,
    "subject_list": subject_lists,
    "ch_info": ch_info,
    "info": "EEG: 1-70filer, 50Hz notch, 256Hz resample;\n\
Subject : 1-30 subject, no 3,9,12,15,16,25 subject; 24 person, 20 trial, 32 EEG channel, 7 pps channels;\n\
Labels :    Arousal: 0:[\"Sadness\", \"Disgust\", \"Neutral\"]、1:[\"Joy, Happiness\", \"Amusement\"] 2:[\"Surprise\", \"Fear\", \"Anger\", \"Anxiety\"]\n\
    Valence: 0:[\"Fear\", \"Anger\", \"Disgust\", \"Sadness\", \"Anxiety\"] 1:[\"Surprise\", \"Neutral\"] 2:[\"Joy, Happiness\", \"Amusement\"]\n\
Eye Track data : DistanceLeft，PupilLeft，ValidityLeft，Distance Right，Pupil Right，Validity Right，Fixation Index，Gaze Point X，Gaze Point Y，Fixation Duration\n\
     PPS data : ECG, GSR，Resp，Temp，Status",
}

hci_data_pkl = "/data/MAHNOB/hci_data.pkl"
joblib.dump(data, hci_data_pkl)

['/data/MAHNOB/hci_data.pkl']